In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('bank.csv')
#df.head(100)
#df.dtypes

In [3]:
df_C=df.select_dtypes(include='object')

In [4]:
rep={'default':{'yes':1,'no':0},'housing':{'yes':1,'no':0},'loan':{'yes':1,'no':0},'y':{'yes':1,'no':0},'month':{'may':5,
'jul':  7,
'aug':  8,
'jun':  6,
'nov':  11,
'apr':  4,
'feb': 2,
'jan':  1,
'oct':   10,
'sep':   9,
'mar':  3,
'dec':12}}
df_C=df_C.replace(rep)
df_C1=df_C.select_dtypes(exclude='object')
df_C2=df_C.select_dtypes(include='object')

In [5]:
df_C2=pd.get_dummies(df_C2, columns=["marital","education","contact","poutcome","job"], prefix=["M","E","C","P","J"])

In [6]:
C=pd.merge(df_C1,df_C2,left_index=True,right_index=True)

In [7]:
df_N=df.select_dtypes(exclude='object')

In [8]:
df_F=pd.merge(df_N,C,left_index=True,right_index=True)

In [9]:
X=df_F.drop('y',axis=1)
y=df_F.y

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y, random_state=42,test_size=.20 )

In [11]:
X_train.drop_duplicates(inplace=True)
X_train.dropna(inplace=True)
X_train_Name=X_train.columns

In [12]:
from sklearn.preprocessing import MinMaxScaler
normal=MinMaxScaler()
X_train_N=pd.DataFrame(normal.fit_transform(X_train))
X_test_N=pd.DataFrame(normal.fit_transform(X_test))
X_train_N.columns=X_train_Name
X_test_N.columns=X_train_Name

In [13]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model=ExtraTreesClassifier()
model.fit(X_train_N,y_train)

ExtraTreesClassifier()

In [14]:
fi=pd.Series(model.feature_importances_,index=X_train_N.columns)
#fi.nlargest(10).plot(kind='barh')
#plt.show()

In [15]:
X_train_N=X_train_N[X_train_N.columns[X_train_N.columns.isin(['previous','housing','pdays',
                               'P_success','campaign','month','day','balance','age','duration'])]]  

X_test_N=X_test_N[X_test_N.columns[X_test_N.columns.isin(['previous','housing','pdays',
                               'P_success','campaign','month','day','balance','age','duration'])]] 

In [16]:
from sklearn import svm
clf=svm.SVC(kernel='rbf',gamma='auto')
clf.fit(X_train_N,y_train)
y_pre=clf.predict(X_test_N)
from sklearn.metrics import classification_report
#print(classification_report(y_test,y_pre))

In [17]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
X_train_smt, y_train_smt = smt.fit_resample(X_train_N, y_train)

In [18]:
clf1=svm.SVC(kernel='rbf', gamma='auto', C=1)
clf1.fit(X_train_smt,y_train_smt)
y_pre_smt=clf1.predict(X_test_N)
#print(classification_report(y_test,y_pre_smt))